In [39]:
%autosave 20

import os
import sys
import random
import numpy as np
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import UpSampling2D
from keras.layers import Concatenate
#from google.colab import drive
import cv2

Autosaving every 20 seconds


In [40]:
'''drive.mount('/content/drive/')
dataset_folder = 'drive/MyDrive/datasets/EndoTect_2020_Segmentation_Test_Dataset'''

def load_data(folder, mask=False):
  imgs = []

  for fname in os.listdir(folder):
    img = []
    if not mask:
      img = cv2.imread(os.path.join(folder, fname))
    else:
      img = cv2.imread(os.path.join(folder, fname), 0)
    if img is not None:
      imgs.append(img)

  return imgs

In [41]:
dataset_folder = 'C:/Users/lesni/Desktop/segmentation dataset/EndoTect_2020_Segmentation_Test_Dataset'

images = load_data(dataset_folder + "/images")
masks = load_data(dataset_folder + "/masks", True)

print(len(images), len(masks))

200 200


In [46]:
for i in range(len(images)):
  images[i] = cv2.resize(images[i], (128,128))
  masks[i] = cv2.resize(masks[i], (128,128))
  mask_ = []
  for j in range(128):
    mask_row = []
    for k in range(128):
      mask_row.append([masks[i][j,k]])
    mask_.append(mask_row)
  mask__ = np.array(mask_)
  masks[i] = mask__

In [47]:
for mask in masks:
  mask_ = np.zeros((128,128,1))
  mask = np.maximum(mask, mask_)

In [48]:
def down_conv(prev_layer, num_layers,  kernel_size=(3,3), padding="same", strides=1, activation="relu"):
  conv = Conv2D(num_layers, kernel_size, padding=padding, strides=strides, activation=activation)(prev_layer)
  conv = Conv2D(num_layers, kernel_size, padding=padding, strides=strides, activation=activation)(conv)
  pool = MaxPool2D((2,2), (2,2))(conv)

  return [conv, pool]

def up_conv(prev_layer, skip_layer, num_layers,  kernel_size=(3,3), padding="same", strides=1, activation="relu"):
  upsamp = UpSampling2D((2,2))(prev_layer)
  conc = Concatenate()([upsamp, skip_layer])
  conv = Conv2D(num_layers, kernel_size, padding=padding, strides=strides, activation=activation)(conc)
  conv = Conv2D(num_layers, kernel_size, padding=padding, strides=strides, activation=activation)(conv)

  return conv

def bootle_neck(prev_layer, num_layers,  kernel_size=(3,3), padding="same", strides=1, activation="relu"):
  conv = Conv2D(num_layers, kernel_size, padding=padding, strides=strides, activation=activation)(prev_layer)
  conv = Conv2D(num_layers, kernel_size, padding=padding, strides=strides, activation=activation)(conv)

  return conv

def reduce_layers(layer):
  return Conv2D(1, (1,1), padding="same", activation="sigmoid")(layer)

In [49]:
input = keras.layers.Input((128,128,3))
conv1, pool1 = down_conv(input, 16)
conv2, pool2 = down_conv(pool1, 32)
conv3, pool3 = down_conv(pool2, 64)
conv4, pool4 = down_conv(pool3, 128)

bneck = bootle_neck(pool4, 256)

up1 = up_conv(bneck, conv4, 128)
up2 = up_conv(up1, conv3, 64)
up3 = up_conv(up2, conv2, 32)
up4 = up_conv(up3, conv1, 16)

output = reduce_layers(up4)

model = keras.models.Model(input, output)

opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d_57 (Conv2D)          (None, 128, 128, 16)         448       ['input_4[0][0]']             
                                                                                                  
 conv2d_58 (Conv2D)          (None, 128, 128, 16)         2320      ['conv2d_57[0][0]']           
                                                                                                  
 max_pooling2d_12 (MaxPooli  (None, 64, 64, 16)           0         ['conv2d_58[0][0]']           
 ng2D)                                                                                      

In [60]:
images = np.array(images)
masks = np.array(masks)
images = images/255.
masks = masks/255.

In [56]:
model.fit(images, masks, epochs=5, validation_split=0.2)

Epoch 1/5
5/5 [==============================] - 13s 3s/step - loss: 144392208.0000 - accuracy: 0.0078 - val_loss: -87033695436800.0000 - val_accuracy: 0.0231
Epoch 2/5
5/5 [==============================] - 14s 3s/step - loss: 1281858.2500 - accuracy: 0.3518 - val_loss: 165532368109568.0000 - val_accuracy: 0.8521
Epoch 3/5
5/5 [==============================] - 13s 3s/step - loss: 12.3093 - accuracy: 0.8512 - val_loss: 605888553418752.0000 - val_accuracy: 0.8588
Epoch 4/5
5/5 [==============================] - 13s 3s/step - loss: 5.3647 - accuracy: 0.8526 - val_loss: 1006779659976704.0000 - val_accuracy: 0.8588
Epoch 5/5
5/5 [==============================] - 12s 2s/step - loss: 4.2138 - accuracy: 0.8526 - val_loss: 1270774254010368.0000 - val_accuracy: 0.8588
